# **第3回 GPUを使用したResNetの転移学習**

「ランタイムのタイプの変更」からGPUを選択し、コードを実行してください。

※上部にある「ドライブにコピー」で自分のドライブにコピーしてから編集・実行してください。

In [ ]:
# ライブラリのインポート
import os
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision.datasets.folder import default_loader
import torchvision.models as models

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# GPUが使用可能か確認
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPUが使用可能なので、GPUを使用します。")
else:
    device = torch.device("cpu")
    print("GPUが使用不可なので、CPUを使用します。")

In [ ]:
# 各種設定
BATCH_SIZE = 32  # バッチサイズ
MAX_EPOCH = 5    # エポック数
IMAGE_SIZE = 224 # 画像サイズ

# シード値の固定
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [3]:
# 自作のデータセットの処理を定義
class MyDataset(Dataset):
	def __init__(self, csv_file, root_dir, transform=None, target_transform=None, loader=default_loader):
		self.df = pd.read_csv(csv_file)          # csvファイルを読み込んでデータフレームとして保存
		self.root_dir = root_dir                 # 画像ファイルが保存されたフォルダのパスを保存
		self.loader = loader                     # 画像を読み込むための関数
		self.transform = transform               # 画像に適用する前処理
		self.target_transform = target_transform # ラベルに適用する前処理

	# データセットのサイズを返す関数
	def __len__(self):
		return len(self.df) 

	# データローダーでバッチを作成するときに使う関数
	def __getitem__(self, idx):
		file_name = self.df.iloc[idx, 0]                  # csvファイルのidx行目にあるfilenameを取得
		img_path = os.path.join(self.root_dir, file_name) # 画像のフルパスを作成（root_dir/filename）
		image = self.loader(img_path)                     # 画像を読み込む
		label = self.df.iloc[idx, 1]                      # csvファイルのidx行目にあるlabelを取得

		# データに前処理を適用
		if self.transform is not None:
				image = self.transform(image)
		if self.target_transform is not None:
				label = self.target_transform(label)

		# 画像，ラベル，画像ファイルのパスを返す
		return image, label, img_path

In [4]:
# データの前処理の方法を定義
transform = transforms.Compose([
	transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # 画像のサイズを統一
	transforms.ToTensor(),                       # データをテンソルに変換
	transforms.Normalize(mean=[0.5, 0.5, 0.5],
						 std=[0.5, 0.5, 0.5])    # 画像のピクセル値の正規化
])

In [5]:
# 訓練データ、検証データ、テストデータの準備

# 訓練データの取得
train_dataset = MyDataset(
	csv_file="/content/drive/MyDrive/jts_ai_seminar_3/data/train.csv", # csvファイルのパス
	root_dir="/content/drive/MyDrive/jts_ai_seminar_3/data/all/",      # 画像が保存されているフォルダ
	transform=transform # データに前処理を適用
)

# 訓練データの一部を検証データとして分割
num_train = len(train_dataset) # 訓練データのデータ数を取得
train_dataset, valid_dataset = torch.utils.data.random_split(
	  train_dataset,
	  [int(num_train * 0.8), int(num_train * 0.2)] # 訓練データの20%を検証データに分割
)

# テストデータの取得
test_dataset = MyDataset(
	csv_file="/content/drive/MyDrive/jts_ai_seminar_3/data/test.csv",
	root_dir="/content/drive/MyDrive/jts_ai_seminar_3/data/all/",
	transform=transform
)

In [6]:
# データローダーの作成
# データセットからバッチを作成してモデルに供給する役割

# 訓練データ用のデータローダー
train_loader = torch.utils.data.DataLoader(
	train_dataset,
	batch_size=BATCH_SIZE, # バッチサイズごとにデータを供給
	shuffle=True,          # 学習時はデータをシャッフル
	pin_memory=True,       # GPU使用時にホストメモリを固定して転送を高速化
    num_workers=2          # データの読み込みを並列処理で高速化するために2つのワーカーを使用
)
# 検証データ用のデータローダー
valid_loader = torch.utils.data.DataLoader(
	valid_dataset,
	batch_size=BATCH_SIZE,
	shuffle=False, # 検証時はシャッフルしない
	pin_memory=True,
    num_workers=2
)
# テストデータ用のデータローダー
test_loader = torch.utils.data.DataLoader(
	test_dataset,
	batch_size=BATCH_SIZE,
	shuffle=False, # テスト時もシャッフルしない
	pin_memory=True,
    num_workers=2
)

In [7]:
# ResNetのインスタンス化（事前学習済みの重みを使用）
model = models.resnet18(pretrained=True)

# すべての層のパラメータを凍結（勾配を計算しないようにする）
for param in model.parameters():
    param.requires_grad = False

# 最終全結合層の再定義
model.fc = nn.Linear(512, 2)

# 最終全結合層のパラメータのみを学習可能に設定（最終層のパラメータの勾配を計算するようにする）
for param in model.fc.parameters():
    param.requires_grad = True

# GPUが使用可能な場合はモデルをGPUに転送
model = model.to(device)

# 損失関数の定義
loss_function = nn.CrossEntropyLoss()

# 最適化手法と学習率の定義
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 学習を通しての損失を記録するためのリスト（学習曲線のプロット用）
train_losses = []
valid_losses = []

# 学習にかかった時間の計測開始
start_time = time.time()

print("\n【学習開始】\n")
print("train_loss: 学習データにおける損失")
print("valid_loss: 検証データにおける損失\n")

# エポックごとの訓練損失と検証損失を出力
print("epoch\ttrain_loss\tvalid_loss")

for epoch in range(MAX_EPOCH):
	model.train()        # モデルを訓練モードに設定
	train_loss_list = [] # 訓練損失を保存するリスト

	# 訓練データでモデルを学習
	for x, label, img_path in train_loader:
		x = x.to(device)                    # GPUが使用可能な場合は画像データをGPUに転送
		label = label.to(device)            # GPUが使用可能な場合は正解ラベルをGPUに転送

		optimizer.zero_grad()               # 勾配をリセット
		output = model(x)                   # モデルの出力を計算
		loss = loss_function(output, label) # 損失を計算
		loss.backward()                     # 勾配を計算
		optimizer.step()                    # 勾配に基づいてパラメータを更新
		train_loss_list.append(loss.item()) # バッチごとの損失を記録

	train_loss_mean = np.mean(train_loss_list) # エポックごとの平均訓練損失
	train_losses.append(train_loss_mean)       # 訓練損失をリストに追加

	model.eval()         # モデルを評価モードに設定
	valid_loss_list = [] # 検証損失を保存するリスト

	# 検証データで損失を計算
	with torch.no_grad():
		for x, label, img_path in valid_loader:
			x = x.to(device)                    # GPUが使用可能な場合は画像データをGPUに転送
			label = label.to(device)            # GPUが使用可能な場合は正解ラベルをGPUに転送

			output = model(x)                   # モデルの出力を計算
			loss = loss_function(output, label) # 損失を計算
			valid_loss_list.append(loss.item()) # バッチごとの損失を記録

	valid_loss_mean = np.mean(valid_loss_list) # エポックごとの平均検証損失
	valid_losses.append(valid_loss_mean)       # 検証損失をリストに追加

	print("{}\t{:.8}\t{:.8}".format(epoch, train_loss_mean, valid_loss_mean))

# 学習にかかった時間の計測終了
end_time = time.time()
print("\n学習にかかった時間 : {} [sec]".format(end_time - start_time))

print("\n【学習終了】\n")

In [ ]:
# 学習曲線のプロット
plt.plot(range(MAX_EPOCH), train_losses, label='Training Loss')
plt.plot(range(MAX_EPOCH), valid_losses, label='Validation Loss', color="green")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.show()

In [ ]:
# テストデータを使ってモデルを評価
model.eval() # モデルを評価モードに設定

# 予測結果とラベルを格納するリスト
all_preds = []
all_labels = []
incorrect_preds = [] # 誤分類された画像を保存するリスト

test_loss = 0
test_correct = 0 # 正解した数
test_total = 0   # 総データ数

# テストデータでの損失と正解率を計算
with torch.no_grad():
	for x, label, img_path in test_loader:
		x = x.to(device)                             # GPUが使用可能な場合は画像データをGPUに転送
		label = label.to(device)                     # GPUが使用可能な場合は正解ラベルをGPUに転送
		
		output = model(x)                            # モデルの出力を計算
		loss = loss_function(output, label)          # 損失を計算
		_, pred = torch.max(output.data, dim=1)      # 出力の最大値を持つクラスを予測
		test_correct += (pred == label).sum().item() # 正解の数をカウント
		test_total += label.size()[0]                # テストデータの総数をカウント

		# バッチ内の各予測とラベルをリストに追加
		all_preds.extend(pred.cpu().numpy())         # 複数の予測ラベルをリストに追加
		all_labels.extend(label.cpu().numpy())       # 実際のラベルも同様に追加

		# バッチ内の誤分類された画像を保存
		for i in range(len(pred)):
			if pred[i] != label[i]:
				incorrect_preds.append((x[i], label[i], pred[i], img_path[i]))


# テストデータにおける正解率を計算して表示
test_accuracy = test_correct / test_total
print("テストデータにおける正解率 : {:.4f}".format(test_accuracy))

In [ ]:
# モデルの保存
model_dir = "/content/drive/MyDrive/jts_ai_seminar_3/model/"
os.makedirs(model_dir, exist_ok=True)
torch.save(model.state_dict(), f"{model_dir}resnet_transfer.pt")

## より詳細な結果の分析

In [ ]:
# 混同行列の計算
cm = confusion_matrix(all_labels, all_preds)

# 混同行列の表示
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['cat', 'dog'], yticklabels=['cat', 'dog'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# ラベルをcatとdogに対応させる関数
def label_to_name(label):
    return "cat" if label == 0 else "dog"

# 誤分類された画像の中からランダムに9個を選択
random_incorrect_preds = random.sample(incorrect_preds, 9)

# 誤分類された画像を表示
plt.figure(figsize=(10, 10))
for i, (image, label, pred, img_path) in enumerate(random_incorrect_preds):  # ランダムに選んだ9個を表示
    image = image.permute(1, 2, 0).cpu().numpy()  # 画像を表示可能な形に変換
    image = (image * 0.5) + 0.5                   # 正規化を元に戻す
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
    plt.title(f'True: {label_to_name(label.item())}, Pred: {label_to_name(pred.item())}')
    plt.axis('off')
plt.tight_layout()
plt.show()